# OCR Constitucion Inga

In [1]:
import os
import re
import PyPDF2
import pandas as pd
from google.cloud import documentai
from google.api_core.client_options import ClientOptions
from typing import Optional, Sequence
from dotenv import load_dotenv

from google.api_core.client_options import ClientOptions
from google.cloud import documentai

load_dotenv()

True

In [2]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = os.getenv('GOOGLE_APPLICATION_CREDENTIALS')
endpoint = 'documentai.googleapis.com'
location = 'us'
project_id = os.getenv('project_id')
processor_id = os.getenv('processor_id')

## Dividir el PDF por partes

In [193]:
entrada_pdf = '/Users/juank/Dropbox/Mac/Desktop/workshop_nlp/data/constitucion_inga.pdf'
salida_pdf = '/Users/juank/Dropbox/Mac/Desktop/workshop_nlp/data/Constitución_inga_scrape.pdf'

In [194]:
cartas = {'carta_gobernadores': {'esp': [62, 62], 'ing': [9, 10]}, 'presidencia': {'esp': [63,63], 'ing': [11,12]}, 
          "carta_traductor": {'esp': [65, 66], 'ing': [13, 15]}, 'favorable': {'esp': [67, 67], 'ing': [16, 17]}, 
          'coordinador': {'esp': [68, 69], 'ing': [18, 20]}, 'titulo': {'esp': [70, 70], 'ing': [21, 21]}}
constitucion = {'esp': [71, 80], 'ing': [[22, 36], [37, 51], [52, 58]]}

In [195]:
def extraer_paginas(entrada_pdf, salida_pdf, paginas):
    with open(entrada_pdf, 'rb') as archivo_entrada:
        lector_pdf = PyPDF2.PdfReader(archivo_entrada)
        
        escritor_pdf = PyPDF2.PdfWriter()
        
        for pagina_num in range (paginas[0], paginas[1] + 1):
            # Las páginas en PyPDF2 comienzan desde 0
            pagina = lector_pdf.pages[pagina_num - 1]
            escritor_pdf.add_page(pagina)
            
        with open(salida_pdf, 'wb') as archivo_salida:
            escritor_pdf.write(archivo_salida)
        return None

In [196]:
def get_pdf_to_text(salida_pdf):
    mime_type = 'application/pdf'   
    client = documentai.DocumentProcessorServiceClient(
    client_options=ClientOptions(api_endpoint=f"{location}-{endpoint}"))
    name = client.processor_path(project_id, location, processor_id)
    with open(salida_pdf, "rb") as image:
        image_content = image.read()
    
    raw_document = documentai.RawDocument(
        content=image_content, mime_type=mime_type)
    
    request = documentai.ProcessRequest(name=name, raw_document=raw_document)
    response = client.process_document(request=request)
    document = response.document
    return document

In [197]:
def layout_to_text(layout: documentai.Document.Page.Layout, text: str) -> str:
    """
    Document AI identifies text in different parts of the document by their
    offsets in the entirety of the document"s text. This function converts
    offsets to a string.
    """
    
    return "".join(
        text[int(segment.start_index) : int(segment.end_index)]
        for segment in layout.text_anchor.text_segments
    )

In [198]:
def extract_text(document, text):
    lista_texto = []
    patron = r"\d+"
    ignore = ['Este texto es una traducción a partir del ikun\n', 'BIENVENIDO ARROYO\nCabildo Gobernador\n', "Emari kasteyano pari ikun re'zagikumana ni\n", 'Los artículos de la Constitución seleccionados para su traducción a la lengua ikun fueron reagrupados de\nacuerdo a distintos temas de interés para la población indígena. Esta reagrupación es el resultado de nuestro\ntrabajo; no corresponde siempre a la división en capitulos de la Constitución.\n']


    for pages in range(0, len(document.pages)):
        for blocks in range(0, len(document.pages[pages].blocks)):
            block = layout_to_text(document.pages[pages].blocks[blocks].layout, text)
            block = re.sub(r'G\.ij\.\.z\.\s(\d+)\.', r'GIJZ\1', block)
            if not re.match(patron, block) and block not in ignore:
                clean_block = [elemento.replace("\n", " ").replace("*", "").strip() for elemento in block.split(".") if elemento != "\n" and len(elemento.replace("\n", " ").replace("*", "").strip().split()) >= 4]
                #clean_block = clean_block.replace("\n", " ").strip()
                lista_texto.extend(clean_block)
    return lista_texto

In [200]:
def extract_tittles(document, text):
    lista_texto = []
    raw_tittles = []
    patron = r'^[IVXLCDM]+\.'
    
    for pages in range(0, len(document.pages)):
        for blocks in range(0, len(document.pages[pages].blocks)):
            block = layout_to_text(document.pages[pages].blocks[blocks].layout, text)

            if re.match(patron, block):
                if block.replace("\n", "").split('.')[1] != '':
                    raw_tittles.append(block)
                    block = re.sub(r'^[IVXLCDM]+\.\s*', '', block)
                    lista_texto.append(block.replace("\n", " ").replace("*", "").lower().strip())
                
                else:
                    raw_tittles.append(block)
                    block = layout_to_text(document.pages[pages].blocks[blocks + 1].layout, text)
                    raw_tittles.append(block)
                    lista_texto.append(block.replace("\n", " ").replace("*", "").lower().strip())
    return lista_texto, raw_tittles

In [201]:
def extract_text_articles_ing (document, text, raw_tittles_ing):
    ignore = ['Digitalizado por Sistema de Bibliotecas-Universidad de los Andes\n']
    patron = r'GIJZ(\d+)'
    pag = r"\d+"
    grupo_parrafos = {}
    texto_ing = ""
    
    for pages in range(0, len(document.pages)):
            for blocks in range(0, lsen(document.pages[pages].blocks)):
                block = layout_to_text(document.pages[pages].blocks[blocks].layout, text)
    
                if block not in ignore and not re.match(pag, block) and block not in raw_tittles_ing:
                    try:
                        # Quitamos comentarios hechos en el texto que no estan en la otra parte
                        index = block.index('*')
                        block = block[:index]
                    except:
                        pass
                    if 'SUIUCHII' in block:
                        index = block.find('SUIUCHII')
                        block = block[index:]
    
                    block = block.replace("\n", " ").replace("*", "")
                    if len(block) != 0:
                        texto_ing += ' ' + block
    
    
    patron_articulo = re.compile(r'SUIUCHII \((\d+)\)\.')
    articulos = patron_articulo.split(texto_ing)[1:]
    
    for i in range(0, len(articulos), 2):
        numero_articulo = int(articulos[i])
        texto_articulo = articulos[i+1].strip()
        oraciones_articulo = [oracion.strip().lower() for oracion in texto_articulo.split('.') if oracion.strip()]
    
        grupo_parrafos[numero_articulo] = oraciones_articulo
    return grupo_parrafos

In [202]:
def extract_text_articles_esp (document, text, raw_tittles_esp):
    patron = r'Artículo (\d+)'
    pag = r"\d+"
    ignore = ['* Los artículos de la Constitución seleccionados para su traducción a la lengua inga fueron reagrupados de\nacuerdo a distintos temas de interés para la población indígena. Esta reagrupación es el resultado de nuestro\ntrabajo; no corresponde siempre a la división en capítulos de la Constitución.\n', '*Estos puntos suspensivos están para representar partes del artículo que no se han traducido, o por ser muy\ntécnicas o por no tener importancia directa para las comunidades.\n', 'Digitalizado por Sistema de Bibliotecas-Universidad de los Andes\n']
    grupo_parrafos = {}
    texto_esp = ""
    for pages in range(0, len(document.pages)):
        for blocks in range(0, len(document.pages[pages].blocks)):
            block = layout_to_text(document.pages[pages].blocks[blocks].layout, text)
    
            if block not in ignore and not re.match(pag, block) and block not in raw_tittles_esp:
                try:
                    # Quitamos comentarios hechos en el texto que no estan en la otra parte
                    index = block.index('*')
                    block = block[:index]
                except:
                    pass
                block = block.replace("\n", " ").replace("*", "")
                if len(block) != 0:
                    texto_esp += ' ' + block
                        
    patron_articulo = re.compile(r'Artículo (\d+)\.')
    articulos = patron_articulo.split(texto_esp)[1:]
    
    # Iterar sobre los artículos y dividirlos en oraciones
    for i in range(0, len(articulos), 2):
        numero_articulo = int(articulos[i])
        texto_articulo = articulos[i+1].strip()
        oraciones_articulo = [oracion.strip().lower() for oracion in texto_articulo.split('.') if oracion.strip()]
    
        grupo_parrafos[numero_articulo] = oraciones_articulo
    return grupo_parrafos

In [203]:
df_final = pd.DataFrame()
for key in cartas.keys():
    paginas_a_extraer = cartas[key]['ing']
    # Llamar a la función para extraer las páginas
    escritor_pdf = extraer_paginas(entrada_pdf, salida_pdf, paginas_a_extraer)
    document_ing = get_pdf_to_text(salida_pdf)
    text_ing = document_ing.text
    lista_ing = extract_text(document_ing, text_ing)

    paginas_a_extraer = cartas[key]['esp']
    # Llamar a la función para extraer las páginas
    escritor_pdf = extraer_paginas(entrada_pdf, salida_pdf, paginas_a_extraer)
    document_esp = get_pdf_to_text(salida_pdf)
    text_esp = document_esp.text
    lista_esp = extract_text(document_esp, text_esp)
    
    # Unir datasets
    traduccion = []
    minimo = min(len(lista_ing), len(lista_esp))
    for i in range(0, minimo):
        traduccion.append({'esp': lista_esp[i], 'ing': lista_ing[i]})        
    df_traduccion = pd.DataFrame(traduccion)
    df_traduccion['parte'] = key
    
    df_final = pd.concat([df_final, df_traduccion], ignore_index=True)
df_final['esp'] = df_final['esp'].str.lower()
df_final['ing'] = df_final['ing'].str.lower()

In [204]:
df_final

,esp,ing,parte
0,carta de los gobernadores ingas,nukanchita sinchi charig taitakuna paikunapa m...,carta_gobernadores
1,"santiago, putumayo, septiembre 26 de 1993","santiago putumayo, septiembre 26 de 1993",carta_gobernadores
2,a los lectores; los gobernadores de los cabild...,kai uraladu kalpachispa makiwa kawachispa chur...,carta_gobernadores
3,anteriormente nunca tuvimos la oportunidad de ...,achka iachag pangata nukanchipa simipi ialichi...,carta_gobernadores
4,los cotraductores que le colaboraron para la t...,"santu dius, ñugpata achka iachag pangata mana ...",carta_gobernadores
...,...,...,...
91,jon landaburu (cnrs) director del centro colom...,ikuti sumaglla tukuspa chi rimaita u rimaikuna...,coordinador
92,"santafé de bogotá, febrero de 1994",imasa nukanchipa wasi kagta,coordinador
93,digitalizado por sistema de bibliotecas-univer...,imasa nukanchita iaia sakiskasina,coordinador
94,artículos de la constitución política de colom...,"mailla nukanchita chaiaskata, nukanchipa rimai...",titulo


In [205]:
df_final.to_csv('/Users/juank/Dropbox/Mac/Desktop/workshop_nlp/data/inga/constitucion_inga_cartas.csv', index=False)

In [206]:
articles_ing = {}
titulos_ing = []
for extract_pages in constitucion['ing']:
    paginas_a_extraer = extract_pages
    # Llamar a la función para extraer las páginas
    escritor_pdf = extraer_paginas(entrada_pdf, salida_pdf, paginas_a_extraer)
    document_ing = get_pdf_to_text(salida_pdf)
    text_ing = document_ing.text
    titulo_ing, raw_tittles_ing = extract_tittles(document_ing, text_ing)
    article_ing = extract_text_articles_ing(document_ing, text_ing, raw_tittles_ing)
    
    titulos_ing.extend(titulo_ing)
    articles_ing.update(article_ing)
    


paginas_a_extraer = constitucion['esp']
# Llamar a la función para extraer las páginas
escritor_pdf = extraer_paginas(entrada_pdf, salida_pdf, paginas_a_extraer)
document_esp = get_pdf_to_text(salida_pdf)
text_esp = document_esp.text
titulos_esp, raw_tittles_esp = extract_tittles(document_esp, text_esp)
articles_esp = extract_text_articles_esp (document_esp, text_esp, raw_tittles_esp)

In [207]:
traduccion = []
for i in range(0, len(titulos_esp)):
    traduccion.append({'esp': titulos_esp[i].strip(), 'ing': titulos_ing[i].strip()})        
df_titulos = pd.DataFrame(traduccion)
df_titulos['parte'] = 'titulos_constitucion'
df_final = pd.concat([df_final, df_titulos], ignore_index=True)

In [208]:
df_titulos

,esp,ing,parte
0,artículos relativos a los principios fundament...,kulumbiamanda achka iachag pangakuna iapa mini...,titulos_constitucion
1,artículos relativos a los derechos de las pers...,nukanchi ima mañangapa ministiska,titulos_constitucion
2,articulos relativos a la protección de los der...,maikan mas suma mañadirukunata kawasunchi,titulos_constitucion
3,artículos relativos a los derechos de las leng...,"nukanchipa simikunamanda, imasa runakunamanda ...",titulos_constitucion
4,artículos relativos a la participación en la vida,nukanchi imasa agllangapa u agllai tukungapa k...,titulos_constitucion
5,artículo relativo a la justicia en los pueblos...,nukanchipa taitakuna imasa,titulos_constitucion
6,artículos relativos a la organización de los t...,nukanchipa alpakuna imasa kangapa kagta,titulos_constitucion
7,artículos relativos a los planes de desarrollo,suma iuiarispa mirachii,titulos_constitucion


In [209]:
traduccion = []
for key in articles_esp.keys():
    try:
        blocks_esp = articles_esp[key]
        blocks_ing = articles_ing[key]
        minimo = min(len(blocks_esp), len(blocks_ing))
        for i in range(0, minimo):
            traduccion.append({'esp': blocks_esp[i], 'ing': blocks_ing[i]})
    except:
        pass
df_articulos = pd.DataFrame(traduccion)
df_articulos['parte'] = 'articulos'
df_final = pd.concat([df_final, df_articulos], ignore_index=True)
df_articulos

,esp,ing,parte
0,"colombia es un estado social de derecho, organ...",nukanchipa atun llagta kulumbia tukuikunamanda...,articulos
1,son fines esenciales del estado: servir a la c...,tukuikunata maki mañachingapami atun llagtapa ...,articulos
2,las autoridades de la república están institui...,"tukuikunamanda ima ministiska maskapungapa, ni...",articulos
3,la soberanía reside exclusivamente en el puebl...,nukanchi imasa munaskasinami iuianchi kaugsang...,articulos
4,el pueblo la ejerce en forma directa o por med...,chi suma iuaikuna nukanchikikin u atun taitaku...,articulos
...,...,...,...
103,"para los efectos de esta participación, la le...","chi kulkikunata chaskingapa, achka iachag pang...",articulos
104,artículo transitorio 45,suiuchiita allichispa kawachig (transitorio 45),articulos
105,los distritos y los municipios percibirán como...,wawa llagtata katiraiagkuna mas achka kulkinmi...,articulos
106,a partir de 1993 entrará a regir lo dispuesto ...,"achka iachag pangakunapa wawami willangapa ká,...",articulos


In [210]:
df_final

,esp,ing,parte
0,carta de los gobernadores ingas,nukanchita sinchi charig taitakuna paikunapa m...,carta_gobernadores
1,"santiago, putumayo, septiembre 26 de 1993","santiago putumayo, septiembre 26 de 1993",carta_gobernadores
2,a los lectores; los gobernadores de los cabild...,kai uraladu kalpachispa makiwa kawachispa chur...,carta_gobernadores
3,anteriormente nunca tuvimos la oportunidad de ...,achka iachag pangata nukanchipa simipi ialichi...,carta_gobernadores
4,los cotraductores que le colaboraron para la t...,"santu dius, ñugpata achka iachag pangata mana ...",carta_gobernadores
...,...,...,...
207,"para los efectos de esta participación, la le...","chi kulkikunata chaskingapa, achka iachag pang...",articulos
208,artículo transitorio 45,suiuchiita allichispa kawachig (transitorio 45),articulos
209,los distritos y los municipios percibirán como...,wawa llagtata katiraiagkuna mas achka kulkinmi...,articulos
210,a partir de 1993 entrará a regir lo dispuesto ...,"achka iachag pangakunapa wawami willangapa ká,...",articulos


In [211]:
df_final.to_csv('/Users/juank/Dropbox/Mac/Desktop/workshop_nlp/data/inga/constitucion_ing_completo.csv', index=False)

In [212]:
df_final

,esp,ing,parte
0,carta de los gobernadores ingas,nukanchita sinchi charig taitakuna paikunapa m...,carta_gobernadores
1,"santiago, putumayo, septiembre 26 de 1993","santiago putumayo, septiembre 26 de 1993",carta_gobernadores
2,a los lectores; los gobernadores de los cabild...,kai uraladu kalpachispa makiwa kawachispa chur...,carta_gobernadores
3,anteriormente nunca tuvimos la oportunidad de ...,achka iachag pangata nukanchipa simipi ialichi...,carta_gobernadores
4,los cotraductores que le colaboraron para la t...,"santu dius, ñugpata achka iachag pangata mana ...",carta_gobernadores
...,...,...,...
207,"para los efectos de esta participación, la le...","chi kulkikunata chaskingapa, achka iachag pang...",articulos
208,artículo transitorio 45,suiuchiita allichispa kawachig (transitorio 45),articulos
209,los distritos y los municipios percibirán como...,wawa llagtata katiraiagkuna mas achka kulkinmi...,articulos
210,a partir de 1993 entrará a regir lo dispuesto ...,"achka iachag pangakunapa wawami willangapa ká,...",articulos
